In [1]:
import numpy as np
import h5py
import argparse
import sys
import re
import codecs

In [63]:
words2index = {}
with open('data/words.dict') as f:
    for line in f:
        (val, key, num) = line.split()
        words2index[key] = int(val)

In [64]:
index2words = {}
with open('data/words.dict') as f:
    for line in f:
        (val, key, num) = line.split()
        index2words[int(val)] = key

In [81]:
def train_get_Ngram(filepath,words2index,N):
    results = []
    with open(filepath) as f:
        for line in f:
            lsplit = [words2index[x] for x in line.split()]
            l = np.append(np.repeat(words2index['<s>'],N-1),lsplit)

            for i in range(len(lsplit)):
                g = l[i:N-1+i]
                v = lsplit[i]
                results.append((g,v))
            results.append((l[-N+1:],words2index['</s>']))
    return results

In [238]:
def valid_test_Ngram(filepath, words2index, N, test = False):
    results = []
    if test == False:
        with open(filepath) as f:
            i=1
            for line in f:
                lsplit = line.split()
                if lsplit[0] == 'Q':
                    topredict = np.array([words2index[x] for x in lsplit[1:]])
                if lsplit[0] == 'C':
                    l = np.append(np.repeat(words2index['<s>'], N-1),[words2index[x] for x in lsplit[1:]])
                    lastNgram =  l[-N+1:] 
                    results.append((lastNgram, topredict))
    else:
        with open(filepath) as f:
            i=1
            for line in f:
                lsplit = line.split()
                if lsplit[0] == 'Q':
                    topredict = np.array([words2index[x] for x in lsplit[1:]])
                if lsplit[0] == 'C':
                    l = np.append(np.repeat(words2index['<s>'], N-1),[words2index[x] for x in lsplit[1:-1]])
                    lastNgram =  l[-N+1:] 
                    results.append((lastNgram, topredict))
    return results

In [198]:
from collections import Counter
def tomatrix(results,train = True):
    N = len(results[0][0])+1
    if train:
        tuplelist = []
        for i in range(len(results)):
            tuplelist.append(tuple(list(np.append(results[i][0],results[i][1]))))
        Count = Counter(tuplelist).most_common()
        tooutput = np.empty((len(Count),N+1))

        for i in range(len(Count)):
            tooutput[i,:] = np.append(np.array(Count[i][0]),Count[i][1])

        return tooutput.astype(int)
    
    else:
        tooutput = np.empty((len(results),50+N-1))
        
        for i in range(len(results)):
            tooutput[i,:] = np.hstack((results[i][1],results[i][0]))
        
        return tooutput.astype(int)

In [82]:
bigrams = train_get_Ngram('data/train.txt',2)

In [83]:
trigrams = train_get_Ngram('data/train.txt',3)

In [250]:
tomatrix(trigrams).shape

(589720, 4)